In [ ]:
# Import libraries.
import pandas as pd

# Settings.
pd.set_option('display.max_colwidth', None)

# Load in data. For now this is the sample generated
# in p2000_brandweer_create_data.r.
scrapes = pd.read_csv("../data/all_sample_scrapes.csv")

# Check load correctly.
scrapes.head()

In [ ]:
# Rename columns.
scrapes.rename(columns={'X1': 'code' ,
                        'X2': 'times',
                        'X3': 'dates',
                        'X4': 'sign',
                        'X5': 'info'},
               inplace=True)

In [ ]:
# Check it worked.
scrapes.columns

In [ ]:
# Count missings.
scrapes.isna().sum()

In [ ]:
# Missings are an artefact of the scrape, separating incidents.
scrapes_nm = scrapes.dropna(inplace = False)

In [ ]:
# Due to scrape frequency, there are loads of duplicates. Remove them.
scrapes_nm_dd = scrapes_nm.drop_duplicates(subset = ['times', 'dates', 'info'],
                                           keep = False)

In [ ]:
# Check it worked. ***Re-check with full data**
raw_dim = scrapes_nm.shape
dd_dim  = scrapes_nm_dd.shape
print(raw_dim, dd_dim)

In [ ]:
# Load in CAP data.
cap = pd.read_csv("../data/capcodelijst_source_in_header.csv", skiprows=1, delimiter=';')

In [ ]:
# Check contents.
cap.head()

In [ ]:
# Join by code. 
scrapes_wcodes = scrapes_nm_dd.merge(cap, on = "code", how = "left")

In [ ]:
# Check structure now.
scrapes_wcodes.head()

In [ ]:
# Create a string-based detection flag and compare.
str(scrapes_wcodes['info']).lower().find('ambulance', 0, len(scrapes_wcodes['info']) )

In [ ]:
# Another way.
if 'ambulance' in str(scrapes_wcodes['info']).lower():
    print("contains ambulance")

In [38]:
# Another way.
scrapes_wcodes['info'].str.contains('Ambulance')

AttributeError: 'Series' object has no attribute 'lower'